In [1]:
import pandas as pd
df = pd.read_csv("data_identification.csv")
df2 = pd.read_csv("emotion.csv")
df3 = pd.read_csv("sampleSubmission.csv")
df4 = pd.read_json("tweets_DM.json", lines=True)
df4['text'] = df4._source.apply(lambda x:x['tweet']['text'])
df4['tweet_id'] = df4._source.apply(lambda x:x['tweet']['tweet_id'])
df4 = df4[['text','tweet_id']]

In [10]:
answer_df = df[df['identification'] == 'test']
answer_df = answer_df.merge(df4,how="inner",on=['tweet_id'])
answer_df = answer_df.drop(columns=['identification'])
x_answer = answer_df['text']

In [2]:
train_df = df[df['identification'] == 'train']
train_df = train_df.merge(df2,how="inner",on=['tweet_id'])
train_df = train_df.merge(df4,how="inner",on=['tweet_id'])
train_df = train_df.drop(columns=['identification'])

In [20]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
vectorizer = TfidfVectorizer(max_features=1000, tokenizer=tknzr.tokenize)  
vectorizer.fit(train_df['text'])

KeyboardInterrupt: 

In [5]:
X_train = train_df['text']
y_train = train_df['emotion']
X_train = vectorizer.transform(X_train)
# X_test = vectorizer.transform(X_test)
X_train = X_train.todense()
# X_test = X_test.todense()
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(y_train.unique()))
y_train = le.transform(y_train)
# y_test = le.transform(y_test)

In [13]:
X_answer = vectorizer.transform(x_answer)
X_answer = X_answer.todense()

In [14]:
import keras

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train_enc = label_encode(label_encoder, y_train)
# I/O check
input_shape = X_train.shape[1]

output_shape = len(label_encoder.classes_)

from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# training setting
epochs = 10
batch_size = 32

# training!
history = model.fit(X_train, y_train_enc, 
                    epochs=epochs, 
                    batch_size=batch_size)

In [ ]:
aa = model.predict(X_answer)
aa = aa.tolist()
y_answer = []
for i in range(len(aa)):
    y_answer.append(aa[i].index(max(aa[i])))

In [15]:
answer_df = answer_df.drop(columns=['text'])
answer_df['emotion'] = list(le.inverse_transform(list(map(int, y_answer))))
answer_df = answer_df.rename(columns={"tweet_id": "id"})
answer_df.to_csv("submission.csv",index=False)

In [85]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train_df['text'], train_df['emotion'], test_size=0.2)

In [115]:
X_train = train_df['text'][0:1000000]
X_test = train_df['text'][1000000:]

In [275]:
y_train = train_df['emotion'][0:1000000]
y_test = train_df['emotion'][1000000:]

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=1000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function word_tokenize at 0x15de84bf8>, use_idf=True,
                vocabulary=None)

In [147]:
import xgboost
params = {}
params['objective'] = 'multi:softmax'
params['eval_metric'] = 'mlogloss'
params['eta'] = 0.04
params['max_depth'] =  6
params['learning_rate'] = 0.1
params['num_class'] = 8

d_train = xgboost.DMatrix(X_train, label=y_train)
d_valid = xgboost.DMatrix(X_test, label=y_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgboost.train(params, d_train, 100, watchlist, early_stopping_rounds=100, verbose_eval=10)
y_pred = bst.predict(xgboost.DMatrix(X_test))

[0]	train-mlogloss:2.00653	valid-mlogloss:2.00692
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 100 rounds.
[10]	train-mlogloss:1.68965	valid-mlogloss:1.68920
[20]	train-mlogloss:1.58177	valid-mlogloss:1.58189
[30]	train-mlogloss:1.52943	valid-mlogloss:1.53059
[40]	train-mlogloss:1.49899	valid-mlogloss:1.50162
[50]	train-mlogloss:1.47886	valid-mlogloss:1.48287
[60]	train-mlogloss:1.46360	valid-mlogloss:1.46914
[70]	train-mlogloss:1.45084	valid-mlogloss:1.45791
[80]	train-mlogloss:1.44009	valid-mlogloss:1.44868
[90]	train-mlogloss:1.43085	valid-mlogloss:1.44079
[99]	train-mlogloss:1.42361	valid-mlogloss:1.43472


In [172]:
from sklearn.metrics import accuracy_score, f1_score
print('Accuracy: %f' % accuracy_score(y_test, y_pred))

Accuracy: 0.488387


In [173]:
y_answer = bst.predict(xgboost.DMatrix(X_answer))

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import pandas as pd
import numpy as np
import warnings

In [47]:
class network(nn.Module):
    
    def __init__(self , num_item):
        
        super().__init__()
        self.fc1 = nn.Linear(num_item , 64)
        self.fc2 = nn.Linear(64 , 64)
        self.out = nn.Linear(64 , 8)
        
    def forward(self , x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.log_softmax(self.out(x))
        return x

In [51]:
model = network(1000).float()
batch_data = TensorDataset(torch.tensor(X_train),torch.tensor(y_train))
dataloader = DataLoader(batch_data, batch_size=128, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=0.001)
for epoch in range(10):
    total_loss = 0 
    for data, target in dataloader:
        # ===================forward====================
        output = model(data.float())
        loss = criterion(output, target)
        total_loss += loss
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, 100, total_loss))


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


epoch [1/100], loss:15885.4053
epoch [2/100], loss:15120.6025
epoch [3/100], loss:14891.4521
epoch [4/100], loss:14777.9502
epoch [5/100], loss:14698.9941
epoch [6/100], loss:14643.4854
epoch [7/100], loss:14603.9678
epoch [8/100], loss:14573.1973
epoch [9/100], loss:14548.4111
epoch [10/100], loss:14530.5029


In [58]:
y_answer = model(torch.tensor(X_answer).float())

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  


AttributeError: 'matrix' object has no attribute 'todense'

Epoch 1/10
45487/45487 [==============================] - 69s 2ms/step - loss: 1.3632 - accuracy: 0.5044
Epoch 2/10
45487/45487 [==============================] - 69s 2ms/step - loss: 1.3115 - accuracy: 0.5220
Epoch 3/10
45487/45487 [==============================] - 70s 2ms/step - loss: 1.2964 - accuracy: 0.5279
Epoch 4/10
45487/45487 [==============================] - 69s 2ms/step - loss: 1.2875 - accuracy: 0.5313
Epoch 5/10
45487/45487 [==============================] - 70s 2ms/step - loss: 1.2817 - accuracy: 0.5332 0s - loss: 1.2817 - 
Epoch 6/10
45487/45487 [==============================] - 69s 2ms/step - loss: 1.2775 - accuracy: 0.5348
Epoch 7/10
45487/45487 [==============================] - 70s 2ms/step - loss: 1.2741 - accuracy: 0.5361
Epoch 8/10
45487/45487 [==============================] - 70s 2ms/step - loss: 1.2714 - accuracy: 0.5375
Epoch 9/10
45487/45487 [==============================] - 71s 2ms/step - loss: 1.2693 - accuracy: 0.5383
Epoch 10/10
45487/45487 [=========